# Chapter6-4 システム構築

## 6-4-1 Webスクレイピング

In [ ]:
from selenium import webdriver

In [ ]:
browser = webdriver.Chrome()

In [ ]:
browser.get('https://sandbox.kikagaku.co.jp/blog/')

In [ ]:
elems = browser.find_elements_by_class_name('stretched-link')

In [ ]:
len(elems)

In [ ]:
for elem in elems:
    print(elem.get_attribute('href'))

In [ ]:
urls = []
for elem in elems:
    url = elem.get_attribute('href')
    urls.append(url)

In [ ]:
urls

In [ ]:
urls = [elem.get_attribute('href') for elem in elems]
urls

In [ ]:
url = elems[0].get_attribute('href')
url

In [ ]:
browser.get(url)

In [ ]:
elem = browser.find_element_by_class_name('newsContent_inner')

In [ ]:
elem.text

In [ ]:
from janome.tokenizer import Tokenizer

In [ ]:
tagger = Tokenizer(wakati=True)

In [ ]:
from gensim import corpora, matutils

In [ ]:
import joblib

In [ ]:
dictionary = joblib.load('dictionary.pkl')
clf = joblib.load('document_classifier.pkl')

In [ ]:
n_words = len(dictionary)
n_words

In [ ]:
def preprocessing(text):
    text = text.replace('\u3000', '')
    text = text.replace('\n', '')
    words = tagger.tokenize(text)
    return words

In [ ]:
words = preprocessing(text)
words[:10]  # 先頭の10件を表示

In [ ]:
def get_bow(words):
    bow_id = dictionary.doc2bow(words)
    bow = matutils.corpus2dense([bow_id], n_words).T
    return bow

In [ ]:
x = get_bow(words)
x

In [ ]:
y = clf.predict(x)[0]
y

In [ ]:
for url in urls:
    browser.get(url)
    text = browser.find_element_by_class_name('newsContent_inner').text
    words = preprocessing(text)
    x = get_bow(words)
    y = clf.predict(x)[0]
    print('URL: {}, Label: {}'.format(url, y))

## 6-4-2 Slack との連携

In [ ]:
slack_url ='<取得したWebhook URL>'

In [ ]:
import requests
import json

In [ ]:
data = json.dumps({
    'username': 'bot',
    'text': 'テスト'
})

In [ ]:
data

In [ ]:
requests.post(slack_url, data=data)

## 6-4-3 文書要約

In [ ]:
url = 'https://sandbox.kikagaku.co.jp/blog/post4.html'
browser.get(url)
title = browser.find_element_by_class_name('blog-post-title').text
text = browser.find_element_by_class_name('newsContent_inner').text

In [ ]:
from summpy.summpy.lexrank import summarize

In [ ]:
# 文章要約の実行
sentences, debug_info = summarize(
    text, sent_limit=3, continuous=True, debug=True
)

In [ ]:
sentences

In [ ]:
text = '''記事のタイトル：
{}

記事のURL：
{}

記事の要約：
・{}
・{}
・{}
'''.format(title, url, sentences[0], sentences[1], sentences[2])

In [ ]:
print(text)

In [ ]:
data = json.dumps({
    'username': 'bot',
    'text': text
})

In [ ]:
requests.post(slack_url, data=data)

In [ ]:
browser.quit()